In [1]:
%reload_ext autoreload
%autoreload 2

# Run HEroBM backmapping #

### 1. Create the inference config file ###

Specify the following parameters:
- mapping: which is the CG mapping to use
- input: input file to backmap. It could be either a pdb, gro or any format compatible with MDAnalysis.
- inputtraj: optional xtc or trr trajectory to load into the input file.
- isatomistic: set to True if the input file is at atomistic resolution. In this case, the input will first be converted to CG, according to the specified mapping, then the model will be used to backmap the CG back again to atomistic resolution. This is used to evaluate model performance.
- selection: optional selection of atoms/beads/residues/molecules to apply on input as a pre-processing
- trajslice: optional indexing of frame using the python slice format [from]:[to][:step]
- model: could be either a deployed model or the .pth model file used for training (usually, '.../run_name/best_model.pth').
- output: Optionally, you can provide a folder where to save the backmapped result

In [1]:
args_dict = {
    "mapping": "martini3",
    "input": "data/tutorial/A2A/md/a2a.pdb",
    "inputtraj": "data/tutorial/A2A/md/a2a.xtc",
    "isatomistic": True, # Set this to False when backmapping actual CG
    "selection": "protein",
    "trajslice": "900:1000:10",
    "model": "results/A2A/A2A.martini3/best_model.pth",
    "output": "backmapped/tutorial/A2A.martini3",
}

args_dict.update({
    "device": "cuda:0",
    "noinvariants": True, # Avoid computing angles and dihedrals when Coarse-graining atomistic input, they are used only for training.
})

### 2. Run backmapping ###

'tolerance' parameter is used as a threshold for energy minimisation.

Note: if you test the A2A model on other proteins, most probably the energy minimised version will have a lower RMSD with respect to the raw backmapped version. This is expected, as the model was trained on a single system and is not yet able to generalise extremely well, thus might create clashes in some cases. Running even mild energy minimisation fixes any clash and gives a sound structure to simulate.

In [ ]:
from herobm.backmapping.hierarchical_backmapping import HierarchicalBackmapping

backmapping = HierarchicalBackmapping(args_dict=args_dict)

backmapped_filenames, backmapped_minimised_filenames, true_filenames, cg_filenames = backmapping.backmap(
    tolerance=500., # Value in Kj/(mol nm). Threshold to stop energy minimisation.
)

After training completion, you will find the results in the './results/tutorial/A2A.martini' folder.

Model weights are saved as 'best_model.pth', while the 'config.yaml' file contains all the directives to load the model, to either perform inference or to continue training/fine-tune the model.

In [ ]:
import MDAnalysis as mda
import nglview as nv
from MDAnalysis.analysis import rms

backmapped_u = mda.Universe(*backmapped_filenames[:1])
ref_u = mda.Universe(*true_filenames[:1])

print("RMSD BB:", rms.rmsd(backmapped_u.select_atoms('name N CA C O').positions, ref_u.select_atoms('name N CA C O').positions, superposition=False))
print("RMSD SC:", rms.rmsd(backmapped_u.select_atoms('protein and not name N CA C O').positions, ref_u.select_atoms('protein and not name N CA C O').positions, superposition=False))

sel = 'protein'
w = nv.show_mdanalysis(backmapped_u)
w.add_representation('licorice', selection='protein')
w

Check reconstructed and original together

In [ ]:
merged_u = mda.Merge(backmapped_u.select_atoms(sel), ref_u.select_atoms(sel))

w = nv.show_mdanalysis(merged_u)
w.add_representation('licorice', selection='protein')
w

Finally, have a look to the minimised structure and compare it to the ground truth

In [ ]:
backmapped_minimised_u = mda.Universe(*backmapped_minimised_filenames[:1])
merged_minimised_u = mda.Merge(backmapped_minimised_u.select_atoms(sel), ref_u.select_atoms(sel))

s = 'name N CA C O and not resname ACE NME'
print("RMSD BB:", rms.rmsd(backmapped_minimised_u.select_atoms(s).positions, ref_u.select_atoms(s).positions, superposition=False))
s = 'protein and not name N CA C O OXT and not type H and not resname ACE NME'
print("RMSD SC:", rms.rmsd(backmapped_minimised_u.select_atoms(s).positions, ref_u.select_atoms(s).positions, superposition=False))

w = nv.show_mdanalysis(merged_minimised_u)
w.add_representation('licorice', selection='protein')
w